In [ ]:
from sklearn import linear_model
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image

# importing the libraries

import pandas as pd
import numpy as np
# for reading and displaying images

import matplotlib.pyplot as plt
%matplotlib inline
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules

import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
"""
    To execute the code run the following command
        python baseline2.py

"""




'\n    To execute the code run the following command\n        python baseline2.py\n\n'

In [ ]:
import torch
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:

root_dir = "/content/drive/My Drive/CIL-project2021/Data/"
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from os import listdir
from os.path import isfile, join

#################################
##### paths to google colab #####
#################################
base_DIR_training = root_dir + "training/training/"
base_DIR_testing = root_dir + "test_images/test_images/"
base_DIR_aug = root_dir + "training_augmented/training_augmented/"

ground_truth_PATH_TRAIN = join(base_DIR_training , "groundtruth") 
images_PATH_TRAIN = join(base_DIR_training , "images")


ground_truth_PATH_AUG = join(base_DIR_aug , "groundtruth") 
images_PATH_AUG = join(base_DIR_aug, "images")

ROTATE = True
TRANSLATE = True
ADD_NOISE = True
BLUR = True

####################################

In [ ]:
import torch
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.9781, 0.2566, 0.4616],
        [0.9977, 0.6183, 0.6016],
        [0.6096, 0.5995, 0.7252],
        [0.6833, 0.5892, 0.8371],
        [0.9938, 0.0723, 0.3763]])


In [ ]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data
def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg


In [ ]:
def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches


In [ ]:
files = os.listdir(ground_truth_PATH_TRAIN)
n = len(files) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(os.path.join(images_PATH_TRAIN, files[i])) for i in range(n) if files[i].endswith('.png')]

#gt_dir = os.path.join(data_dir,  ground_truth_PATH_TRAIN)
print("Loading " + str(n) + " images")
gt_imgs = [load_image(os.path.join(ground_truth_PATH_TRAIN, files[i])) for i in range(n) if files[i].endswith('.png')]
print(files[0])

n = 100 # Only use 10 images for training


Loading 100 images
Loading 100 images
satImage_063.png


In [ ]:
#print('Image size = ' + str(imgs[0].shape[0]) + ',' + str(imgs[0].shape[1]))
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]


print("img_patches: ", np.array(img_patches).shape )
# Linearize list of patches
img_patches = np.asarray([img_patches[i][j][:,:,0] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
print("img_patches: ", img_patches.shape)
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])


img_patches:  (100, 625, 16, 16, 3)
img_patches:  (62500, 16, 16)


In [ ]:
# Extract 6-dimensional features consisting of average RGB color as well as variance
def extract_features(img):
    feat_m = np.mean(img, axis=(0,1))
    feat_v = np.var(img, axis=(0,1))
    feat = np.append(feat_m, feat_v)
    return feat

# Extract 2-dimensional features consisting of average gray color as well as variance
def extract_features_2d(img):
    #print("small image dim: ", img.shape)
    feat_m = np.mean(img)
    feat_v = np.var(img)
    feat = np.append(feat_m, feat_v)
    return feat

# Extract features for a given image
def extract_img_features(filename):
    img = load_image(filename)
    #print("img size: ", img.shape)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([(img_patches[i][:,:,0]) for i in range(len(img_patches))])
    return X


# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch


In [ ]:
def value_to_class(v):
    df = np.sum(v)
    #print("df: ", df)
    if df > foreground_threshold:
        return 1
    else:
        return 0

#X = np.asarray([extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
X = img_patches 
#print("the shape of X: ", X.shape)
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])


In [ ]:
X2= X.reshape(62500, 1, 16, 16)
print("shape of X: ", X2.shape)

print("shape of Y: ", Y.shape)

shape of X:  (62500, 1, 16, 16)
shape of Y:  (62500,)


In [ ]:

train_x  = torch.from_numpy(X2)

# converting the target into torch format
train_y =  Y.astype(int)
train_y = torch.from_numpy(Y)
train_x.shape, train_y.shape

(torch.Size([62500, 1, 16, 16]), torch.Size([62500]))

In [ ]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(64, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        print(x.size())
        x = self.linear_layers(x)
        return x
    


In [ ]:
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
        
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(train_x), Variable(train_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [ ]:
# defining the number of epochs
n_epochs = 5
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

torch.Size([62500, 64])
torch.Size([62500, 64])
Epoch :  1 	 loss : tensor(2.5912, grad_fn=<NllLossBackward>)
torch.Size([62500, 64])
torch.Size([62500, 64])
torch.Size([62500, 64])
torch.Size([62500, 64])
Epoch :  3 	 loss : tensor(0.5960, grad_fn=<NllLossBackward>)
torch.Size([62500, 64])
torch.Size([62500, 64])
torch.Size([62500, 64])
torch.Size([62500, 64])
Epoch :  5 	 loss : tensor(0.5759, grad_fn=<NllLossBackward>)


In [ ]:
# prediction for training set

output = model(train_x)
print(output.shape)
    
#softmax = torch.exp(output).cpu()
#prob = list(softmax.numpy())

m = Softmax(dim=1)
out = m(output)
print("out: ",out.shape)
predictions = np.argmax(list(out.detach().numpy()), axis=1)


# accuracy on training set
accuracy_score(train_y, predictions)

torch.Size([62500, 64])
torch.Size([62500, 10])
out:  torch.Size([62500, 10])


0.73584

In [ ]:
def prediction_to_file(idimg, res, submission_file):
  predictions = res.reshape(38,38)
  for j in range(0, predictions.shape[1]):
    for i in range(0, predictions.shape[0]):
      with open(submission_file, 'a') as f:
        _id = '{:03d}'.format(idimg)
        f.write(f'{_id}_{j*patch_size}_{i*patch_size},{predictions[i,j]}\n')


ids = [int(filename.split('_')[1].split('.')[0]) for filename in os.listdir(base_DIR_testing)]
#print("ids: ", ids)
ids.sort()
submission_file = '/content/drive/My Drive/CIL-project2021/submission_baseline1.csv'
try:
  os.remove(submission_file)
except OSError:
  pass
with open(submission_file, 'a') as f:
  f.write('id,prediction\n')

print("predicting...")
for id in ids:
  #for path in files:
  Xtest = extract_img_features(f'{base_DIR_testing}/test_{id}.png')
  print(Xtest.shape)
  Xtest2= Xtest.reshape(1444,1, 16, 16)
  test_x  = torch.from_numpy(Xtest2)
  print(test_x.shape)

  output_test = model(test_x)
  m = Softmax(dim=1)
  out_test = m(output_test)
  print("out: ", out_test.shape)
  predictions_test = np.argmax(list(out_test.detach().numpy()), axis=1)
  #print("the shape of Xtest: ", Xtest.shape)
  #print("the shape of res: ", res.shape)
  #plt.scatter(Xtest[:, 0], Xtest[:, 1], c=res, edgecolors='k', cmap=plt.cm.Paired)
  prediction_to_file(id, predictions_test, submission_file)

 

predicting...
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
torch.Size([1444, 1, 16, 16])
torch.Size([1444, 64])
out:  torch.Size([1444, 10])
(1444, 16, 16)
t